In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from chainer.datasets import tuple_dataset


all = pd.read_table('./train/train_labels.txt',header=None)


allData = []
ss =all.iloc[:,0]
for i in range(len(all)):
    ss =all.iloc[:,0][i]
    ss =ss.split(' ')
    allData.append(ss)


imageData = []
labelData = []
for pathAndLabel in allData:
    img = Image.open('./train/images/'+pathAndLabel[0])
    #3チャンネルの画像をr,g,bそれぞれの画像に分ける
    r,g,b = img.split()
    rImgData = np.asarray(np.float32(r)/255.0)
    gImgData = np.asarray(np.float32(g)/255.0)
    bImgData = np.asarray(np.float32(b)/255.0)
    imgData = np.asarray([rImgData, gImgData, bImgData])
    imageData.append(imgData)
    labelData.append(np.int32(pathAndLabel[1]))
    train = tuple_dataset.TupleDataset(imageData, labelData)

In [2]:
list(train)[0]

(array([[[0.43529412, 0.40784314, 0.3764706 , ..., 0.54901963,
          0.54509807, 0.5411765 ],
         [0.44705883, 0.41960785, 0.38039216, ..., 0.54509807,
          0.54509807, 0.5411765 ],
         [0.4627451 , 0.42352942, 0.39215687, ..., 0.54509807,
          0.54509807, 0.5411765 ],
         ...,
         [0.34509805, 0.34509805, 0.34509805, ..., 0.25490198,
          0.30980393, 0.34509805],
         [0.34509805, 0.34509805, 0.34509805, ..., 0.24705882,
          0.24705882, 0.23137255],
         [0.34117648, 0.34509805, 0.34509805, ..., 0.5058824 ,
          0.30980393, 0.20392157]],
 
        [[0.4392157 , 0.4117647 , 0.38431373, ..., 0.5372549 ,
          0.53333336, 0.5294118 ],
         [0.4509804 , 0.42352942, 0.3882353 , ..., 0.53333336,
          0.53333336, 0.5294118 ],
         [0.46666667, 0.42745098, 0.4       , ..., 0.53333336,
          0.53333336, 0.5294118 ],
         ...,
         [0.3647059 , 0.3647059 , 0.3647059 , ..., 0.        ,
          0.01960784, 0.

In [3]:
import pandas as pd
import numpy as np
from PIL import Image
from chainer.datasets import tuple_dataset


all = pd.read_table('./valid/valid_labels.txt',header=None)


allData = []
ss =all.iloc[:,0]
for i in range(len(all)):
    ss =all.iloc[:,0][i]
    ss =ss.split(' ')
    allData.append(ss)


imageData = []
labelData = []
for pathAndLabel in allData:
    img = Image.open('./valid/images/'+pathAndLabel[0])
    #3チャンネルの画像をr,g,bそれぞれの画像に分ける
    r,g,b = img.split()
    rImgData = np.asarray(np.float32(r)/255.0)
    gImgData = np.asarray(np.float32(g)/255.0)
    bImgData = np.asarray(np.float32(b)/255.0)
    imgData = np.asarray([rImgData, gImgData, bImgData])
    imageData.append(imgData)
    labelData.append(np.int32(pathAndLabel[1]))
    val = tuple_dataset.TupleDataset(imageData, labelData)

In [6]:
import chainer
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import chainer.functions as F           # パラメータを含まない関数を収録したパッケージ
import chainer.links as L
import pandas as pd




class VGG16Model(chainer.Chain):

    def __init__(self, out_size):
        super(VGG16Model, self).__init__(
            base = L.VGG16Layers(),
            fc = L.Linear(None, out_size)
        )

    def __call__(self, x):
        h = self.base(x, layers=['fc7'])
        y = self.fc(h['fc7'])
        return y




model = L.Classifier(VGG16Model(out_size=10))
optimizer = chainer.optimizers.Adam(alpha=1e-4)
optimizer.setup(model)

model.predictor.base.disable_update()

gpu = 0
if gpu >= 0:
    chainer.cuda.get_device(gpu).use()
    model.to_gpu(gpu)










# Training parameter
epoch = 50
batchsize = 4096  # バッチサイズは大きめに設定

from chainer import training
from chainer.training import extensions

# Iterator
train_iter = chainer.iterators.SerialIterator(train, batchsize)
val_iter = chainer.iterators.SerialIterator(val, batchsize, repeat=False, shuffle=False)


# Set up a trainer
updater = training.updaters.ParallelUpdater(train_iter, optimizer, devices=gpu)
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result')

trainer.extend(extensions.Evaluator(val_iter, model, device=gpu))  # Evaluatorはどれかmainなどで使っているGPUを指定すればよい

# Reduce the learning rate by half every 25 epochs.
trainer.extend(extensions.ExponentialShift('lr', 0.5), trigger=(25, 'epoch'))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot at each epoch
trainer.extend(extensions.snapshot(), trigger=(epoch, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

trainer.extend(extensions.PrintReport(
    ['epoch',  'elapsed_time', 'lr', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy',]))

# Print a progress bar to stdout
# trainer.extend(extensions.ProgressBar())

trainer.run()


AttributeError: 'int' object has no attribute 'keys'